In [45]:
import os
import json
import requests
from json_repair import repair_json

In [46]:
# ——————————加载环境变量——————————
from dotenv import load_dotenv
load_dotenv()

True

In [47]:
api_url = os.getenv("LLM_URL")
api_key = os.getenv("LLM_API_KEY")
model = os.getenv("LLM_MODEL")

In [48]:
def chat(messages):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": messages
    }

    response = requests.post(api_url, headers=headers, json=data)

    answer = response.json()["choices"][0]["message"]["content"] 
    
    return answer

1 用户输入

In [49]:
user_input = '有没有便宜的套餐，来个20g套餐以上的，5块钱以上的，按照流量大小排序'

2 NLU语义理解(需要请求llm)

In [50]:
objective='''你的任务是识别用户对手机流量套餐产品的选择条件。
'''

In [51]:
context='''每种流量套餐产品包含三个属性：名称(name)，月费价格(price)，月流量(data)。
根据用户输入，识别用户在上述三种属性上的倾向。
'''

In [52]:
outputs='''以JSON格式输出。
1. name字段的取值为string类型，取值必须为以下之一：经济套餐、畅游套餐、无限套餐、校园套餐 或 null；

2. price字段的取值为一个结构体 或 null，包含两个字段：
(1) operator, string类型，取值范围：'<='（小于等于）, '>=' (大于等于), '=='（等于）
(2) value, int类型

3. data字段的取值为取值为一个结构体 或 null，包含两个字段：
(1) operator, string类型，取值范围：'<='（小于等于）, '>=' (大于等于), '=='（等于）
(2) value, int类型或string类型，string类型只能是'无上限'

4. 用户的意图可以包含按price或data排序，以sort字段标识，取值为一个结构体：
(1) 结构体中以"ordering"="descend"表示按降序排序，以"value"字段存储待排序的字段
(2) 结构体中以"ordering"="ascend"表示按升序排序，以"value"字段存储待排序的字段

输出中只包含用户提及的字段，不要猜测任何用户未直接提及的字段，不输出值为null的字段。
'''

In [53]:
example='''便宜的套餐：{"sort":{"ordering"="ascend","value"="price"}}
有没有不限流量的：{"data":{"operator":"==","value":"无上限"}}
流量大的：{"sort":{"ordering"="descend","value"="data"}}
100G以上流量的套餐最便宜的是哪个：{"sort":{"ordering"="ascend","value"="price"},"data":{"operator":">=","value":100}}
月费不超过200的：{"price":{"operator":"<=","value":200}}
就要月费180那个套餐：{"price":{"operator":"==","value":180}}
经济套餐：{"name":"经济套餐"}

客服：有什么可以帮您
用户：100G套餐有什么

{"data":{"operator":">=","value":100}}

客服：有什么可以帮您
用户：100G套餐有什么
客服：我们现在有无限套餐，不限流量，月费300元
用户：太贵了，有200元以内的不

{"data":{"operator":">=","value":100},"price":{"operator":"<=","value":200}}

客服：有什么可以帮您
用户：便宜的套餐有什么
客服：我们现在有经济套餐，每月50元，10G流量
用户：100G以上的有什么

{"data":{"operator":">=","value":100},"sort":{"ordering"="ascend","value"="price"}}

客服：有什么可以帮您
用户：100G以上的套餐有什么
客服：我们现在有畅游套餐，流量100G，月费180元
用户：流量最多的呢

{"sort":{"ordering"="descend","value"="data"},"data":{"operator":">=","value":100}}
'''

In [54]:
#缺失用户输入的d_prompt
'''
1目标objective
2描述context
3输出格式outputs
4常见案例example
5用户输入inputs
'''
d_prompt=f'''
        {objective}
        {context}

        输出格式如下：
        {outputs}

        例如：
        {example}

        以下是需要处理的内容：
        __INPUT__
'''
#input还没有定义，所以用字符串先替代

In [55]:
#用户的输入替换添加到最后的__INPUT__中
prompt=d_prompt.replace("__INPUT__",user_input)

In [56]:
def chat(messages):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": messages
    }

    response = requests.post(api_url, headers=headers, json=data)

    answer = response.json()["choices"][0]["message"]["content"] 
    
    return answer

In [57]:
def send(prompt):
    # answer = client.chat.completions.create(
    #         model="gpt-3.5-turbo",
    #         messages=[{"role": "user", "content":prompt}],
    #         temperature=0.3,
    #     )
    # response=answer.choices[0].message.content # 正常回答，str的json格式
    # response = chat(prompt)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    messages=[{"role": "user", "content":prompt}]

    data = {
        "model": model,
        "messages": messages,
        "response_format": { "type": "json_object" } # 强制返回json格式
    }

    r = requests.post(api_url, headers=headers, json=data)
    print(r)

    response = r.json()["choices"][0]["message"]["content"] 



    print(response)
    print(type(response))

    # 修复json格式
    nlu=json.loads(repair_json(response)) # 解析成dict字典格式
    print(nlu)
    print(type(nlu))

    return { k:v for k,v in nlu.items() if v }#dict字典格式
    #作用：除去值为空的键值对
    #k:v 表示新字典中的键值对。
    #for k,v in nlu.items() 表示遍历字典nlu中的键值对，并将键赋值给变量k，值赋值给变量v。
    #if v 表示筛选条件，只有当值v非空（即不为False、0、空字符串等）时，才会保留该键值对。

In [58]:
nlu=send(prompt)

<Response [200]>

{"sort":{"ordering":"ascend","value":"data"},"data":{"operator":">=","value":20},"price":{"operator":">=","value":5}}

<class 'str'>
{'sort': {'ordering': 'ascend', 'value': 'data'}, 'data': {'operator': '>=', 'value': 20}, 'price': {'operator': '>=', 'value': 5}}
<class 'dict'>


In [59]:
# 1按照需求重要程度排序
# 2并记录了每个需求的要求
nlu

{'sort': {'ordering': 'ascend', 'value': 'data'},
 'data': {'operator': '>=', 'value': 20},
 'price': {'operator': '>=', 'value': 5}}

3 DST状态更新

In [60]:
#用来存储当前的筛选条件
state={}

In [61]:
#如果用户已经制定了套餐名称，则直接清空所有已经存在的筛选条件
if "name" in nlu:
    state.clear()

print(state)

{}


In [62]:
#如果排序两者同样重要，就删掉不处理了
if "sort" in nlu:#检查字典中是否包含sort键
    slot = nlu["sort"]["value"]#取出字典中sort对应的值
    if slot in state and state[slot]["operator"] == "==": # 如果其对应操作为==
        del state[slot] # 则删除sort的值

print(state)

{}


In [63]:
#以dict形式写入到state中
for k, v in nlu.items():
    print(k)
    print(v)
    state[k] = v
    #return state

print(state)

sort
{'ordering': 'ascend', 'value': 'data'}
data
{'operator': '>=', 'value': 20}
price
{'operator': '>=', 'value': 5}
{'sort': {'ordering': 'ascend', 'value': 'data'}, 'data': {'operator': '>=', 'value': 20}, 'price': {'operator': '>=', 'value': 5}}


4 policy对话策略(筛选：embedding，或知识图谱，输出符合的套餐列表)

In [64]:
#数据库，embedding，或知识图谱
data = [
            {"name":"经济套餐","price":50,"data":10,"requirement":None},
            {"name":"畅游套餐","price":180,"data":100,"requirement":None},
            {"name":"无限套餐","price":300,"data":1000,"requirement":None},
            {"name":"校园套餐","price":150,"data":200,"requirement":"在校生"},
        ]

In [65]:
#需要推荐的产品
recommend=[]

In [66]:
#一条一条对比是否符合筛选条件
#条件筛选，默认是true，如果条件不合适就返回false，过滤掉
for r in data:
    select = True
    if r["requirement"]:
        if "status" not in state or state["status"]!=r["requirement"]:
            continue
    for k, v in state.items():
        if k == "sort":
            continue
        if k == "data" and v["value"] == "无上限":
            if r[k] != 1000:
                select = False
                break
        if "operator" in v:
            if not eval(str(r[k])+v["operator"]+str(v["value"])):
                select = False
                break
        elif str(r[k])!=str(v):
            select = False
            break
    if select:
        recommend.append(r)

print(recommend)

[{'name': '畅游套餐', 'price': 180, 'data': 100, 'requirement': None}, {'name': '无限套餐', 'price': 300, 'data': 1000, 'requirement': None}]


In [67]:
#排序
key = "price"
reverse = False
if "sort" in state:
    key = state["sort"]["value"]
    reverse = state["sort"]["ordering"] == "descend"
a= sorted(recommend,key=lambda x: x[key] ,reverse=reverse)
print(a)

[{'name': '畅游套餐', 'price': 180, 'data': 100, 'requirement': None}, {'name': '无限套餐', 'price': 300, 'data': 1000, 'requirement': None}]


5 NLG对话生成(需要请求llm)

In [68]:
#a是需要推荐的内容的json格式
#user_input是用户的输入
#把用户需求，和向用户介绍的套餐结合到一起
def _wrap(user_input,records):

    prompt_templates = {
    "recommand" : "用户说：__INPUT__ \n\n向用户介绍如下产品：__NAME__，月费__PRICE__元，每月流量__DATA__G。",
    "not_found" : "用户说：__INPUT__ \n\n没有找到满足__PRICE__元价位__DATA__G流量的产品，询问用户是否有其他选择倾向。"
}
    if records:
        prompt = prompt_templates["recommand"].replace("__INPUT__",user_input)
        r = records[0]
        for k,v in r.items():
            prompt = prompt.replace(f"__{k.upper()}__",str(v))
    else:
        prompt = prompt_templates["not_found"].replace("__INPUT__",user_input)
        for k,v in state.items():
            if "operator" in v:
                prompt = prompt.replace(f"__{k.upper()}__",v["operator"]+str(v["value"]))
            else:
                prompt = prompt.replace(f"__{k.upper()}__",str(v))
    return prompt

In [69]:
x=_wrap(user_input,a)
x

'用户说：有没有便宜的套餐，来个20g套餐以上的，5块钱以上的，按照流量大小排序 \n\n向用户介绍如下产品：畅游套餐，月费180元，每月流量100G。'

6 语言模型输出

In [70]:
import copy
# 再添加预设提示词进行回答
def call( prompt):
    memory= [
        {
            "role": "system","content": "你是一个手机流量套餐的客服代表，你叫客服kk。可以帮助用户选择最合适的流量套餐产品。"
        }
    ]
    memory = copy.deepcopy(memory)
    memory.append({"role": "user", "content": prompt})
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": memory
    }

    response = requests.post(api_url, headers=headers, json=data)

    answer = response.json()["choices"][0]["message"]["content"] 
    
    return answer

In [71]:
answer=call(x)

In [72]:
answer

'当然可以，我们这里有一个非常划算的套餐推荐给您。畅游套餐，月费只需要180元，每月可以享受高达100G的流量。这个套餐不仅流量充足，而且价格也非常实惠。如果您需要的是20G以上的流量套餐，并且月费在5元以上的话，这个畅游套餐绝对是一个不错的选择。您可以考虑一下这个套餐，如果有任何其他问题或者需要进一步的帮助，请随时告诉我。'